In [1]:
from supabase import create_client, Client
import os

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

In [2]:
import pandas as pd

In [3]:
response = (
    supabase.table("Payments")
    .select("*")
    .csv()
    .execute()
)

In [4]:
response.data

'id,created_at,uuid,quantity,price,purchase_date,tags\n5922,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,2,3,2025-04-01,"{еда,молочные}"\n5923,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,1,1,2025-04-01,"{еда,хлебобулочные}"\n5924,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,3,2,2025-04-02,"{фрукты,съедобное}"\n5925,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,1,3,2025-04-03,"{еда,молочные}"\n5926,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,2,1,2025-04-04,"{еда,хлебобулочные}"\n5927,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,5,2,2025-04-05,"{фрукты,съедобное}"\n5928,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,1,5,2025-04-06,"{еда,молочные}"\n5929,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3ab5d61a,10,1,2025-04-07,"{овощи,съедобное}"\n5930,"2025-04-12 20:21:04.018982+00",47823327-2b0f-48c8-9513-614c3a

In [5]:
from io import StringIO

In [6]:
df = pd.read_csv(StringIO(response.data), index_col=0, parse_dates=['purchase_date'])
df.drop(columns=['created_at'], inplace=True)
df

,uuid,quantity,price,purchase_date,tags
id,,,,,
5922,47823327-2b0f-48c8-9513-614c3ab5d61a,2,3,2025-04-01,"{еда,молочные}"
5923,47823327-2b0f-48c8-9513-614c3ab5d61a,1,1,2025-04-01,"{еда,хлебобулочные}"
5924,47823327-2b0f-48c8-9513-614c3ab5d61a,3,2,2025-04-02,"{фрукты,съедобное}"
5925,47823327-2b0f-48c8-9513-614c3ab5d61a,1,3,2025-04-03,"{еда,молочные}"
5926,47823327-2b0f-48c8-9513-614c3ab5d61a,2,1,2025-04-04,"{еда,хлебобулочные}"
5927,47823327-2b0f-48c8-9513-614c3ab5d61a,5,2,2025-04-05,"{фрукты,съедобное}"
5928,47823327-2b0f-48c8-9513-614c3ab5d61a,1,5,2025-04-06,"{еда,молочные}"
5929,47823327-2b0f-48c8-9513-614c3ab5d61a,10,1,2025-04-07,"{овощи,съедобное}"
5930,47823327-2b0f-48c8-9513-614c3ab5d61a,1,6,2025-04-08,"{еда,напитки}"


In [7]:
df.dtypes

uuid                     object
quantity                  int64
price                     int64
purchase_date    datetime64[ns]
tags                     object
dtype: object

In [8]:
test_uuid = "47823327-2b0f-48c8-9513-614c3ab5d61a"

In [9]:
tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
tmp = tmp.groupby(['product_name', 'purchase_date']).sum().reset_index().sort_values(by='purchase_date')

tmp

KeyError: "['product_name'] not in index"

In [ ]:
tmp['purchase_date'].unique().tolist()

[Timestamp('2025-04-01 00:00:00'),
 Timestamp('2025-04-02 00:00:00'),
 Timestamp('2025-04-03 00:00:00'),
 Timestamp('2025-04-04 00:00:00'),
 Timestamp('2025-04-05 00:00:00'),
 Timestamp('2025-04-06 00:00:00'),
 Timestamp('2025-04-07 00:00:00'),
 Timestamp('2025-04-08 00:00:00'),
 Timestamp('2025-04-09 00:00:00'),
 Timestamp('2025-04-11 00:00:00'),
 Timestamp('2025-04-12 00:00:00'),
 Timestamp('2025-04-13 00:00:00'),
 Timestamp('2025-04-14 00:00:00'),
 Timestamp('2025-04-15 00:00:00'),
 Timestamp('2025-04-16 00:00:00'),
 Timestamp('2025-04-17 00:00:00'),
 Timestamp('2025-04-18 00:00:00'),
 Timestamp('2025-04-19 00:00:00'),
 Timestamp('2025-04-20 00:00:00'),
 Timestamp('2025-04-21 00:00:00'),
 Timestamp('2025-04-22 00:00:00'),
 Timestamp('2025-04-23 00:00:00'),
 Timestamp('2025-04-24 00:00:00'),
 Timestamp('2025-04-25 00:00:00'),
 Timestamp('2025-04-26 00:00:00'),
 Timestamp('2025-04-27 00:00:00'),
 Timestamp('2025-04-28 00:00:00'),
 Timestamp('2025-04-29 00:00:00'),
 Timestamp('2025-04-

In [ ]:
import pandas as pd
import json

# Предположим, что tmp уже существует
# tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
# tmp = tmp.groupby(['product_name', 'purchase_date']).sum()

# Сброс индекса, чтобы сделать product_name и purchase_date обычными колонками
tmp_reset = tmp.reset_index()

# Создание списка уникальных дат
labels = tmp_reset['purchase_date'].astype(str).unique().tolist()

# Подготовка данных для Quantity
quantity_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    quantity_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['quantity'].reindex(labels, fill_value=0).tolist()
    })

# Подготовка данных для Price
price_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    price_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['price'].reindex(labels, fill_value=0).tolist()
    })

# Формирование финального JSON
chart_data = {
    "quantity": {
        "labels": labels,
        "datasets": quantity_datasets
    },
    "price": {
        "labels": labels,
        "datasets": price_datasets
    }
}

# Вывод JSON в читаемом формате
print(json.dumps(chart_data, indent=4))

{
    "quantity": {
        "labels": [
            "2025-04-01",
            "2025-04-02",
            "2025-04-03",
            "2025-04-04",
            "2025-04-05",
            "2025-04-06",
            "2025-04-07",
            "2025-04-08",
            "2025-04-09",
            "2025-04-11",
            "2025-04-12",
            "2025-04-13",
            "2025-04-14",
            "2025-04-15",
            "2025-04-16",
            "2025-04-17",
            "2025-04-18",
            "2025-04-19",
            "2025-04-20",
            "2025-04-21",
            "2025-04-22",
            "2025-04-23",
            "2025-04-24",
            "2025-04-25",
            "2025-04-26",
            "2025-04-27",
            "2025-04-28",
            "2025-04-29",
            "2025-04-30"
        ],
        "datasets": [
            {
                "label": "\u041a\u0430\u0440\u0442\u043e\u0444\u0435\u043b\u044c",
                "data": [
                    0,
                    0,
     

In [13]:
# user = pd.read_csv('data/user_1.csv', parse_dates=['real_transaction_dttm'])
# user['real_transaction_dttm'] = user['real_transaction_dttm'].dt.date
# user['loyalty_cashback_category_nm'] = user['loyalty_cashback_category_nm'].apply(lambda x: '{' + str(x) + '}')
# user = user.drop(columns=['account_rk', 'transaction_type_cd'])
# user.columns = ["price", "purchase_date", "tags"]
# user['price'] = user['price'].astype(int)
# user.sample(100).to_csv('data/user_1_capypay.csv', index=False)
# user.head()

In [26]:
df.head()

,uuid,product_name,quantity,price,purchase_date,tags
id,,,,,,
5825,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,3,2025-04-01,"{еда,молочные}"
5826,47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,1,1,2025-04-01,"{еда,хлебобулочные}"
5827,47823327-2b0f-48c8-9513-614c3ab5d61a,Яблоки,3,2,2025-04-02,"{фрукты,съедобное}"
5828,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,1,3,2025-04-03,"{еда,молочные}"
5829,47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,2,1,2025-04-04,"{еда,хлебобулочные}"


In [28]:
tmp = df[df['uuid'] == test_uuid][['quantity', 'price', 'tags', 'purchase_date']]
tmp['tags'] = tmp['tags'].str.strip('{}').str.split(',')
tmp = tmp.explode('tags')

tmp.groupby(['tags', 'purchase_date']).sum().reset_index()

,tags,purchase_date,quantity,price
0,еда,2025-04-01,3,4
1,еда,2025-04-03,1,3
2,еда,2025-04-04,3,2
3,еда,2025-04-06,1,5
4,еда,2025-04-08,1,6
...,...,...,...,...
56,хлебобулочные,2025-04-01,1,1
57,хлебобулочные,2025-04-04,3,2
58,хлебобулочные,2025-04-16,3,1
59,хлебобулочные,2025-04-22,2,1


In [23]:
tmp = df[df['uuid'] == test_uuid][['quantity', 'price', 'tags']]
tmp['tags'] = tmp['tags'].str.strip('{}').str.split(',')
tmp = tmp.explode('tags').groupby('tags').sum().reset_index()

tmp

,tags,quantity,price
0,еда,36,73
1,молочные,19,43
2,напитки,7,24
3,овощи,36,5
4,съедобное,63,17
5,фрукты,27,12
6,хлебобулочные,10,6


In [29]:
df.head()

,uuid,product_name,quantity,price,purchase_date,tags
id,,,,,,
5825,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,3,2025-04-01,"{еда,молочные}"
5826,47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,1,1,2025-04-01,"{еда,хлебобулочные}"
5827,47823327-2b0f-48c8-9513-614c3ab5d61a,Яблоки,3,2,2025-04-02,"{фрукты,съедобное}"
5828,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,1,3,2025-04-03,"{еда,молочные}"
5829,47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,2,1,2025-04-04,"{еда,хлебобулочные}"


In [30]:
tmp = df[df['uuid'] == test_uuid][['price', 'purchase_date']]

tmp.groupby(['purchase_date']).sum().reset_index()

,purchase_date,price
0,2025-04-01,4
1,2025-04-02,2
2,2025-04-03,3
3,2025-04-04,2
4,2025-04-05,2
5,2025-04-06,5
6,2025-04-07,1
7,2025-04-08,6
8,2025-04-09,3
9,2025-04-11,2
